In [6]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

23/09/02 09:09:46 WARN Utils: Your hostname, Yinkas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.51 instead (on interface en0)
23/09/02 09:09:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/02 09:09:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
from pyspark.sql.functions import isnull, col

In [9]:
from pyspark.sql.types import IntegerType

In [10]:
rows = [('John', 19), ('Paul', 18)]
df = spark.createDataFrame(rows, schema='Name string, Age int')

In [11]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)



In [12]:
df.show()

+----+---+
|Name|Age|
+----+---+
|John| 19|
|Paul| 18|
+----+---+



In [13]:
df.write.json('result')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/yinka/PycharmProjects/pythonProject5/result already exists. Set mode as "overwrite" to overwrite the existing path.

In [14]:
#read the csv
workers_df = spark.read.csv('/Users/yinka/PycharmProjects/pythonProject5/venv/workers.csv', header=True)

In [15]:
workers_df.show()

+--------+----------+------+---+----------+--------------------+
|   name1|     name2|gender|age|  language|          occupation|
+--------+----------+------+---+----------+--------------------+
| Adolpho|  Pendrich|Female| 21| Norwegian|                null|
|    Lief|      Ughi|Female| 22|Portuguese|  Recruiting Manager|
|    Alvy|Malatalant|  Male| 24| Malayalam|       VP Accounting|
|     Syd|   Daniels|Female| 28|  Moldovan|     Staff Scientist|
| Roselin|   Girkins|Female| 35|   Amharic|   Financial Analyst|
| Pamella|    Pablos|  Male| 33|  Georgian|  Research Assistant|
|  Harmon|Sandercock|  Male| 23|   English|   Marketing Manager|
|   Louis| Waterland|Female| 21|    Telugu|Computer Systems ...|
| Yehudit|   Oxenden|  Male| 35|   Spanish|  Speech Pathologist|
|  Karrie|   Thomsen|  Male| 18|     Tetum|        Statistician|
|  Marven| Burchnall|Female| 18|   English| Executive Secretary|
|   Nerta|   Kensley|  Male| 19|   Bosnian|               Nurse|
|   Piper|   Yarnold|  Ma

In [16]:
workers_df.describe().show()

23/09/02 07:48:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----+----------+------+-----------------+--------+--------------------+
|summary|name1|     name2|gender|              age|language|          occupation|
+-------+-----+----------+------+-----------------+--------+--------------------+
|  count|   50|        50|    50|               50|      50|                  46|
|   mean| null|      null|  null|            25.62|    null|                null|
| stddev| null|      null|  null|5.275627441163675|    null|                null|
|    min|Abbot|Abramovicz|Female|               18| Amharic|Accounting Assistant|
|    max| Zora|   Yarnold|  Male|               35|    Zulu|       Web Developer|
+-------+-----+----------+------+-----------------+--------+--------------------+



In [17]:
workers_df.filter(isnull(workers_df.occupation)).show()

+-------+----------+------+---+---------+----------+
|  name1|     name2|gender|age| language|occupation|
+-------+----------+------+---+---------+----------+
|Adolpho|  Pendrich|Female| 21|Norwegian|      null|
|  Piper|   Yarnold|  Male| 21|   Nepali|      null|
|   Jase| Devonside|Female| 29|    Dutch|      null|
|  Leroy|Abramovicz|  Male| 19|   Kazakh|      null|
+-------+----------+------+---+---------+----------+



In [18]:
workers_df.filter(col('gender') == 'Male').show()

+--------+----------+------+---+----------+--------------------+
|   name1|     name2|gender|age|  language|          occupation|
+--------+----------+------+---+----------+--------------------+
|    Alvy|Malatalant|  Male| 24| Malayalam|       VP Accounting|
| Pamella|    Pablos|  Male| 33|  Georgian|  Research Assistant|
|  Harmon|Sandercock|  Male| 23|   English|   Marketing Manager|
| Yehudit|   Oxenden|  Male| 35|   Spanish|  Speech Pathologist|
|  Karrie|   Thomsen|  Male| 18|     Tetum|        Statistician|
|   Nerta|   Kensley|  Male| 19|   Bosnian|               Nurse|
|   Piper|   Yarnold|  Male| 21|    Nepali|                null|
|Mallorie|   Kelledy|  Male| 30|    German|     Legal Assistant|
|  Dorice|   Glidder|  Male| 21|   English|    Internal Auditor|
|   Kelsy|    Sacase|  Male| 23| Icelandic|Senior Financial ...|
|   Lindi|     Gouly|  Male| 28|   Kannada|   Software Engineer|
|  Prisca|   Canning|  Male| 32|    Fijian|VP Product Manage...|
|   Tadio|      Renn|  Ma

In [ ]:
workers_df.filter((col('language').isin('Spanish', 'English')) & (col('age') < 30)).show()

In [19]:
workers_df.dtypes

[('name1', 'string'),
 ('name2', 'string'),
 ('gender', 'string'),
 ('age', 'string'),
 ('language', 'string'),
 ('occupation', 'string')]

In [20]:
workers_df = workers_df.withColumn('age', col('age').cast(IntegerType()))

In [21]:
workers_df.dtypes

[('name1', 'string'),
 ('name2', 'string'),
 ('gender', 'string'),
 ('age', 'int'),
 ('language', 'string'),
 ('occupation', 'string')]

In [22]:
workers_df = workers_df.withColumnsRenamed({'name1': 'firstname', 'name2': 'lastname'})

In [23]:
workers_df = workers_df.na.fill('Not Provided')

In [24]:
#using sql
workers_df.createOrReplaceTempView('workers')
spark.sql('SELECT * FROM workers').show()


+---------+----------+------+---+----------+--------------------+
|firstname|  lastname|gender|age|  language|          occupation|
+---------+----------+------+---+----------+--------------------+
|  Adolpho|  Pendrich|Female| 21| Norwegian|        Not Provided|
|     Lief|      Ughi|Female| 22|Portuguese|  Recruiting Manager|
|     Alvy|Malatalant|  Male| 24| Malayalam|       VP Accounting|
|      Syd|   Daniels|Female| 28|  Moldovan|     Staff Scientist|
|  Roselin|   Girkins|Female| 35|   Amharic|   Financial Analyst|
|  Pamella|    Pablos|  Male| 33|  Georgian|  Research Assistant|
|   Harmon|Sandercock|  Male| 23|   English|   Marketing Manager|
|    Louis| Waterland|Female| 21|    Telugu|Computer Systems ...|
|  Yehudit|   Oxenden|  Male| 35|   Spanish|  Speech Pathologist|
|   Karrie|   Thomsen|  Male| 18|     Tetum|        Statistician|
|   Marven| Burchnall|Female| 18|   English| Executive Secretary|
|    Nerta|   Kensley|  Male| 19|   Bosnian|               Nurse|
|    Piper

In [25]:
query = """SELECT language, COUNT(firstname) AS total_speakers FROM workers GROUP BY language ORDER BY total_speakers DESC
"""
spark.sql(query).show()

+----------+--------------+
|  language|total_speakers|
+----------+--------------+
|   English|             5|
|  Croatian|             3|
|    Kazakh|             3|
|Portuguese|             2|
| Norwegian|             2|
|   Amharic|             2|
|    Fijian|             1|
|Papiamento|             1|
|    Nepali|             1|
|   Kannada|             1|
|   Finnish|             1|
|     Tetum|             1|
| Icelandic|             1|
|  Assamese|             1|
|   Bosnian|             1|
|   Guaraní|             1|
|    Telugu|             1|
|  Filipino|             1|
|     Azeri|             1|
|   Spanish|             1|
+----------+--------------+
only showing top 20 rows



In [26]:
query = """SELECT CONCAT(firstname, " ", lastname) AS fullname, date_sub(curdate(), (age * 365)) AS DOB, * FROM workers
"""

In [27]:
workers_df = spark.sql(query)

In [28]:
workers_df.show()

+-----------------+----------+---------+----------+------+---+----------+--------------------+
|         fullname|       DOB|firstname|  lastname|gender|age|  language|          occupation|
+-----------------+----------+---------+----------+------+---+----------+--------------------+
| Adolpho Pendrich|2002-09-07|  Adolpho|  Pendrich|Female| 21| Norwegian|        Not Provided|
|        Lief Ughi|2001-09-07|     Lief|      Ughi|Female| 22|Portuguese|  Recruiting Manager|
|  Alvy Malatalant|1999-09-08|     Alvy|Malatalant|  Male| 24| Malayalam|       VP Accounting|
|      Syd Daniels|1995-09-09|      Syd|   Daniels|Female| 28|  Moldovan|     Staff Scientist|
|  Roselin Girkins|1988-09-10|  Roselin|   Girkins|Female| 35|   Amharic|   Financial Analyst|
|   Pamella Pablos|1990-09-10|  Pamella|    Pablos|  Male| 33|  Georgian|  Research Assistant|
|Harmon Sandercock|2000-09-07|   Harmon|Sandercock|  Male| 23|   English|   Marketing Manager|
|  Louis Waterland|2002-09-07|    Louis| Waterland

In [29]:
workers_df.select(['fullname', 'gender', 'age', 'language', 'occupation', 'DOB'])

DataFrame[fullname: string, gender: string, age: int, language: string, occupation: string, DOB: date]

In [30]:
workers_df.write.option('header', True).partitionBy(['language', 'gender']).mode('overwrite').format('csv').save('result')

In [31]:
query = """SELECT CONCAT(firstname, " ", lastname) AS fullname, date_sub(curdate(), (age * 365)) AS DOB, * ,year(date_sub(curdate(), (age * 365))) AS YearOfBirth FROM workers
"""

In [32]:
workers_df.select(['fullname', 'gender', 'age', 'language', 'occupation', 'DOB', 'YearOfBirth']).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `YearOfBirth` cannot be resolved. Did you mean one of the following? [`DOB`, `workers`.`age`, `fullname`, `workers`.`firstname`, `workers`.`gender`].;
'Project [fullname#731, gender#669, age#646, language#670, occupation#671, DOB#732, 'YearOfBirth]
+- Project [concat(firstname#667,  , lastname#668) AS fullname#731, date_sub(current_date(Some(America/Regina)), (age#646 * 365)) AS DOB#732, firstname#667, lastname#668, gender#669, age#646, language#670, occupation#671]
   +- SubqueryAlias workers
      +- View (`workers`, [firstname#667,lastname#668,gender#669,age#646,language#670,occupation#671])
         +- Project [coalesce(firstname#654, cast(Not Provided as string)) AS firstname#667, coalesce(lastname#653, cast(Not Provided as string)) AS lastname#668, coalesce(gender#65, cast(Not Provided as string)) AS gender#669, age#646, coalesce(language#67, cast(Not Provided as string)) AS language#6